# Shapefile

1. daytime - decsending
2. get 5 meg shapefile file
3. understand shapefiles
4. geohash path rows from [path row list]


In [1]:
# get shapefile

! mkdir -p /data/shape


In [2]:
# ! wget https://landsat.usgs.gov/sites/default/files/documents/WRS2_descending.zip -O /data/shape/WRS2_descending.zip

In [3]:
! ls /data/shape


In [4]:
#! (cd /data/shape && unzip ./WRS2_descending.zip)

In [5]:
! ls /data/shape

# The Python Part 

In [6]:
%matplotlib inline

import pandas as pd
import geopandas

pd.options.display.max_rows = 10

In [7]:
wrs_things = geopandas.read_file("zip:/data/shape/WRS2_descending.zip")

DriverError: '/vsizip//data/shape/WRS2_descending.zip' does not exist in the file system, and is not recognized as a supported dataset name.

In [ ]:
wrs_things.head()

https://medium.com/dunder-data/selecting-subsets-of-data-in-pandas-6fcd0170be9c

In [ ]:
mywrs = wrs_things[['PATH', 'ROW', 'geometry']]

In [ ]:
mywrs.head()

In [ ]:
wrs = mywrs.loc['171':'172']

In [ ]:
wrs.head()

In [ ]:
wrs = mywrs[(mywrs['PATH'] >= 170) & (mywrs['PATH'] <= 173) & (mywrs['ROW'] >= 60) & (mywrs['ROW'] <= 63)]

In [ ]:
wrs.head()

In [ ]:
wrs.plot()

In [ ]:
import folium
rwanda_map = folium.Map(location=[-1.0, 32.0],
                   tiles='Mapbox Bright', zoom_start=7)

In [ ]:
rwanda_map

In [ ]:
p172061 = wrs[(wrs['PATH']==172) & (wrs['ROW']==61)]

In [ ]:
p172061.plot()

In [ ]:
p172061.geometry

In [ ]:
p172061['geometry']

In [ ]:
crs = {'init': 'epsg:4326'}
poly = geopandas.GeoDataFrame(index=[0], crs=crs, geometry=p172061['geometry'])

In [ ]:
my_useful_geojson = geopandas.GeoSeries(p172061['geometry']).to_json()
my_useful_geojson

In [ ]:
rwanda_map = folium.Map(location=[-1.0, 32.0],
                   tiles='Mapbox Bright', zoom_start=7)

In [ ]:
folium.GeoJson(my_useful_geojson).add_to(rwanda_map)

In [ ]:
rwanda_map

In [ ]:
# https://python-visualization.github.io/folium/quickstart.html

In [ ]:
for path in range(172,174):
    print(path)
    for row in range (61,64):
        print (path, row)

In [ ]:
def display_wrs(wrs, path, row, scene_count=1):
    #print(path,row)
    this_path_row = wrs[(wrs['PATH']==path) & (wrs['ROW']==row)]
    if not this_path_row.empty:
        print(path,row, this_path_row)
        my_useful_geojson = geopandas.GeoSeries(this_path_row['geometry']).to_json()
    # print(my_useful_geojson)
        folium.GeoJson(my_useful_geojson).add_to(rwanda_map)
        my_point = this_path_row['geometry'].centroid.values
        useful_point = [my_point[0].bounds[1], my_point[0].bounds[0]]
        print(useful_point)
        #print(dir(my_point[0].bounds))
        popup=str(path) + '/' + str(row) + "--" + str(scene_count)
        folium.Marker(location=useful_point, popup=popup).add_to(rwanda_map)

In [ ]:
for path in range(172,174):
    print(path)
    for row in range (61,64):
        print (path, row)
        display_wrs(wrs, path, row)

In [ ]:
rwanda_map

In [ ]:
import geohash

print(help(geohash.encode))

In [ ]:
def get_ul(wrs, path, row):
    this_path_row = wrs[(wrs['PATH']==path) & (wrs['ROW']==row)]
    if not this_path_row.empty:
        # print(this_path_row.bounds)
        #print(type(this_path_row))
        bds = this_path_row.bounds
        # print(type(bds))
        # print(bds['minx'].values[0])
        ul = [bds['minx'].values[0], bds['maxy'].values[0]]
        # print(ul)
        gh = geohash.encode(ul[1], ul[0], precision=3)
        line = str(path) +',' + str(row) + ',' + gh
        print(line)
        mylist = [path, row, gh]
        return path,row,gh
        
        

In [ ]:
COLUMN_NAMES = ['PATH', 'ROW', 'GEOHASH']
df = pd.DataFrame(columns=COLUMN_NAMES)


linelist = []
geo = []


for path in range(172,174):
    #print(path)
    for row in range (61,64):
        #print (path, row)
        path,row,gh = get_ul(wrs, path, row)
        linelist = [path, row, gh]
        geo.append(linelist)
        

#geo = [pathlist, rowlist, ghlist]
#geo = [(172, 172, 172, 173, 173, 173), (61, 62, 63, 61, 62, 63), ('kxv', 'kxt', 'kxk', 'kxu', 'kxe', 'kx7')]

print (geo)
df = pd.DataFrame.from_records(geo, columns=COLUMN_NAMES)

df

In [ ]:
sales = [('Jones LLC', 150, 200, 50),
         ('Alpha Co', 200, 210, 90),
         ('Blue Inc', 140, 215, 95)]
labels = ['account', 'Jan', 'Feb', 'Mar']
df = pd.DataFrame.from_records(sales, columns=labels)

df

In [ ]:
geo = [(172, 172, 172, 173, 173, 173), (61, 62, 63, 61, 62, 63), ('kxv', 'kxt', 'kxk', 'kxu', 'kxe', 'kx7')]

In [ ]:
# Elasticsearch geohash bucket counts

In [ ]:
from elasticsearch_dsl.connections import connections
from elasticsearch_dsl import Search, Q

# Define a default Elasticsearch client
client = connections.create_connection(hosts=['http://elastic:9200'])
s = Search(using=client, index="datacube", doc_type="rwanda")

# how simple we just past CURL code here
body = {
    "aggregations" : {
        "tony" : {
            "geohash_grid" : {
                "field" : "ul", 
                "precision" : 3
            }
        },
        "aggs": {
                "geo_bounds": {
                    "field": "ul"
                }
        }
    }
}

s = Search.from_dict(body)
s = s.index("datacube")
s = s.doc_type("rwanda")
body = s.to_dict()

t = s.execute()

geo_dict = {}
for item in t.aggregations.tony.buckets:
# item.key will the house number
    print (item.key, item.doc_count)
    geo_dict[item.key] = item.doc_count

In [ ]:
geo_dict

In [ ]:
COLUMN_NAMES = ['PATH', 'ROW', 'GEOHASH', 'SCENE_COUNT']
df = pd.DataFrame(columns=COLUMN_NAMES)


linelist = []
geo = []

for path in range(172,174):
    #print(path)
    for row in range (61,64):
        #print (path, row)
        path,row,gh = get_ul(wrs, path, row)
        if gh in geo_dict:
            scene_count = geo_dict[gh]
        else:
            scene_count = 0
        linelist = [path, row, gh, scene_count]
        geo.append(linelist)
        

#geo = [pathlist, rowlist, ghlist]
#geo = [(172, 172, 172, 173, 173, 173), (61, 62, 63, 61, 62, 63), ('kxv', 'kxt', 'kxk', 'kxu', 'kxe', 'kx7')]

print (geo)
df = pd.DataFrame.from_records(geo, columns=COLUMN_NAMES)

df

In [ ]:
def get_table_row(table, path, row):
    this_path_row = table[(table['PATH']==path) & (table['ROW']==row)]
    if not this_path_row.empty:
        print(this_path_row)
        mylist = this_path_row.values[0]
        return mylist[0], mylist[1], mylist[2], mylist[3]

In [ ]:
path,row,gh,scene_count = get_table_row(df, 172, 61)
print(path)

In [ ]:
for path in range(172,174):
    print(path)
    for row in range (61,64):
        print (path, row)
        path,row,gh,scene_count = get_table_row(df, path, row)
        display_wrs(wrs, path, row, scene_count)

In [ ]:
rwanda_map

In [ ]:
import folium
rwanda_map = folium.Map(location=[-1.0, 32.0],
                   tiles='OpenStreetMap', zoom_start=7)

In [ ]:
for path in range(172,174):
    print(path)
    for row in range (61,64):
        print (path, row)
        path,row,gh,scene_count = get_table_row(df, path, row)
        display_wrs(wrs, path, row, scene_count)

In [ ]:
rwanda_map